In [6]:
import tensorflow as tf
from tensorflow.keras.utils import image_dataset_from_directory
from sklearn.metrics import confusion_matrix, precision_score, recall_score
import seaborn as sns
import matplotlib.pyplot as plt
import numpy as np
import os
# import pandas as pd

### Load dataset

In [4]:
batch_size = 32
img_height = 184
img_width = 216
img_size = (img_height, img_width)
img_shape = img_size + (3,)
ds_path = "/drive0-storage/Gracia/dataset_1"

with tf.device("CPU"):
    test_ds = image_dataset_from_directory(ds_path,
                                        seed = 123,
                                        image_size = img_size,
                                        batch_size = batch_size)


Found 21408 files belonging to 3 classes.


In [5]:
class_names = test_ds.class_names
num_classes = len(class_names)

print(class_names)

['bipolar_disorder', 'healthy_controls', 'schizophrenia']


### Load Model

In [7]:
model = tf.keras.models.load_model('./models/cnn3_20epoch.h5')

### Testing

In [9]:
result = model.evaluate(test_ds)
dict(zip(model.metrics_names, result))

2023-06-10 10:44:00.474658: I tensorflow/stream_executor/cuda/cuda_dnn.cc:424] Loaded cuDNN version 8600


  1/669 [..............................] - ETA: 32:53 - loss: 0.4353 - accuracy: 0.7812

2023-06-10 10:44:01.773556: I tensorflow/stream_executor/cuda/cuda_blas.cc:1633] TensorFloat-32 will be used for the matrix multiplication. This will only be logged once.


669/669 [==============================] - 41s 58ms/step - loss: 0.3102 - accuracy: 0.9005


{'loss': 0.3101620674133301, 'accuracy': 0.9004577994346619}

In [ ]:
predicted = []  # predicted labels
true = []  # true labels

for image_batch, label_batch in test_dataset:  
    true.append(label_batch)
    prediction = model.predict(image_batch, verbose=0)
    predicted.append(np.argmax(prediction, axis=-1))

# convert labels into tensors
true_labels = tf.concat([item for item in true], axis=0)
predicted_labels = tf.concat([item for item in predicted], axis=0)

cf_matrix = confusion_matrix(true_labels, predicted_labels, normalize='true')

# plot confusion  matrix
plt.figure(figsize = (8,5))
sns.heatmap(cf_matrix, 
            annot=True)
plt.title('Confusion Matrix', size=15, fontweight='bold')
plt.show()

In [ ]:
precision = precision_score(true_labels, predicted_labels, average='weighted')
sensitivity = recall_score(true_labels, predicted_labels, average='weighted')
specificity = []
for i in range(num_classes):
    true_negatives = np.sum(np.delete(np.delete(cf_matrix, i, axis=0), i, axis=1))
    false_positives = np.sum(cf_matrix[:, i]) - cf_matrix[i, i]
    specificity.append(true_negatives / (true_negatives + false_positives))

# Calculate average specificity
average_specificity = np.mean(specificity)

print("Accuracy:", result[1])
print("Precision:", precision)
print("Sensitivity:", sensitivity)
print("Specificity:", average_specificity)